# Make Production Run List

__Description__: Calculates the discharge for each annual exceedance probability (AEP) within the weights table using the mean flow frequency curve. 

__Input__: The mean flow frequency curve table and the annual exceedance probability weights table.

__Output__: A table with the annual exceedance probability, discharge, and weight.

---
## Load Libraries, Parameters, and Data:
### Libraries:

In [1]:
import sys
sys.path.append('../../core')
from meanffc import*

### Parameters and Data:
#### Site specific (papermill):

##### Make directory if it does not exist:

In [3]:
make_directories([outputs_dir], verbose)

C:\Users\sputnam\Desktop\MeanFFC\Outputs - already exists


##### Convert each dictionary to a pandas dataframe:
##### Mean flow frequency curve:

In [4]:
mean_curve = pd.DataFrame.from_dict(mean_curve) 
mean_curve.head(2)

,Q_Mean_cfs,Q_Median_cfs
0.9,1431.9,1433.3
0.5,2095.3,2083.5


##### Annual exceedance probability weights:

In [5]:
AEP_Weights = pd.DataFrame.from_dict(AEP_Weights) 
AEP_Weights.head(2)

,AEP,Weight
E0001,0.482369,0.035594
E0002,0.448352,0.032774


---
## Calculate the Discharge for Each AEP:
### Initialize a Table with the AEPs, Z-Variate of the AEPs, and Recurrence Intervals:

In [6]:
AEP = list(AEP_Weights['AEP'])

AEPz = zvar(AEP) 

table = ffc_summary(AEP, AEPz, True, verbose) 

,AEPz,RI
AEP,,
0.482369,0.044208,2.073100
0.448352,0.129827,2.230391


### Extract the AEP and Mean/Median Discharge from the Mean Flow Frequency Table:

In [7]:
AEPm = [float(aep) for aep in mean_curve.index] 

AEPmz = zvar(AEPm) 

Qmean = np.log10(mean_curve['Q_Mean_cfs'])

Qmedian = np.log10(mean_curve['Q_Median_cfs'])

### Interpolate the Discharge and Add the Weight Column:

In [8]:
table['Q_Mean_cfs'] = np.round(10**table['AEPz'].apply(interp_q(AEPmz, Qmean)), round_decimals)

table['Q_Median_cfs'] = np.round(10**table['AEPz'].apply(interp_q(AEPmz, Qmedian)), round_decimals)

table = AEP_Weights.join(table, on = 'AEP')

table.head(2)

,AEP,Weight,AEPz,RI,Q_Mean_cfs,Q_Median_cfs
E0001,0.482369,0.035594,0.044208,2.073100,2127.0,2115.0
E0002,0.448352,0.032774,0.129827,2.230391,2189.6,2177.4


---
## Save the Results:

In [9]:
PRL = table.drop(columns=['AEPz']) 

outputs_dir = pl.Path(outputs_dir)

PRL.to_csv(outputs_dir/'ProductionRunList_{0}.csv'.format(gage_ID))

sb.glue('ProRunList', PRL.to_dict())

PRL.head(2)

,AEP,Weight,RI,Q_Mean_cfs,Q_Median_cfs
E0001,0.482369,0.035594,2.073100,2127.0,2115.0
E0002,0.448352,0.032774,2.230391,2189.6,2177.4


---
## End